In [14]:
import glob
import torch
import torch.nn as nn

import importlib
import utils.bert
import utils.tokenizer
import utils.mlm_dataset
import utils.config

importlib.reload(utils.bert)
importlib.reload(utils.tokenizer)
importlib.reload(utils.mlm_dataset)
importlib.reload(utils.config)

from torch.utils.data import DataLoader
from utils.tokenizer import Tokenizer
from utils.mlm_dataset import MLMDataset
from utils.bert import MiniBERT
from utils.config import Config

In [ ]:
def read_in_chunks(file, chunk_size=10_000):
    chunk = []
    for line in file:
        chunk.append(line)
        if len(chunk) == chunk_size:
            yield " ".join(chunk)
            chunk = []
    if chunk:
        yield " ".join(chunk)


files = glob.glob("data/Persian-WikiText-*.txt")
assert len(files) > 0, "No data files found!"

In [16]:
tokenizer = Tokenizer(vocab_size=Config.vocab_size)

print("🔹 Building vocabulary on full dataset (chunk-based)...")
for path in files:
    with open(path, encoding="utf-8") as f:
        for chunk in read_in_chunks(f):
            tokenizer.add_texts([chunk])

tokenizer.finalize_vocab()
print(f"✅ Vocab size: {len(tokenizer.word2id)}")

🔹 Building vocabulary on full dataset (chunk-based)...
✅ Vocab size: 30000


In [ ]:

dataset = MLMDataset(
    file_paths=files,
    tokenizer=tokenizer,
    max_len=Config.max_len,
    mlm_prob=0.15
)

loader = DataLoader(
    dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MiniBERT(Config).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-100)


batch = next(iter(loader))

input_ids = batch["input_ids"].to(device)
labels = batch["labels"].to(device)
attention_mask = batch["attention_mask"].to(device)


In [20]:

logits = model(input_ids, attention_mask)

print("Logits shape:", logits.shape)

loss = criterion(
    logits.view(-1, Config.vocab_size),
    labels.view(-1)
)

print("Initial loss:", loss.item())

Logits shape: torch.Size([16, 128, 30000])
Initial loss: 10.459650039672852


In [21]:
from tqdm import tqdm
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=Config.lr)

model.train()

for epoch in range(Config.epochs):
    total_loss = 0
    progress = tqdm(loader, desc=f"Epoch {epoch+1}")

    for batch in progress:
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)

        loss = criterion(
            logits.view(-1, Config.vocab_size),
            labels.view(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    print(f"✅ Epoch {epoch+1} | Avg Loss: {avg_loss:.4f}")

    torch.save(
        model.state_dict(),
        f"mini_bert_epoch_{epoch+1}.pt"
    )

Epoch 1: 100%|██████████| 48374/48374 [1:10:32<00:00, 11.43it/s, loss=4.42]  


✅ Epoch 1 | Avg Loss: 4.5435


Epoch 2: 100%|██████████| 48374/48374 [1:16:01<00:00, 10.60it/s, loss=3.41]  


✅ Epoch 2 | Avg Loss: 3.6395


Epoch 3: 100%|██████████| 48374/48374 [1:21:17<00:00,  9.92it/s, loss=3.66]  


✅ Epoch 3 | Avg Loss: 3.3982


In [22]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm


def evaluate_mlm(model, dataloader, criterion, device):
    model.eval()

    all_preds = []
    all_labels = []

    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            logits = model(input_ids, attention_mask)

            loss = criterion(
                logits.view(-1, logits.size(-1)),
                labels.view(-1)
            )
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)

            mask = labels != -100
            masked_preds = preds[mask]
            masked_labels = labels[mask]

            all_preds.extend(masked_preds.cpu().numpy())
            all_labels.extend(masked_labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")

    return avg_loss, acc, f1

In [23]:
test_loss, test_acc, test_f1 = evaluate_mlm(
    model,
    loader,          
    criterion,
    device
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)
print("Test F1:", test_f1)

Evaluating: 100%|██████████| 48374/48374 [44:20<00:00, 18.18it/s]    


Test Loss: 3.0940938827669844
Test Accuracy: 0.47689607258775435
Test F1: 0.17536173481619854


In [26]:
%matplotlib inline

import matplotlib.pyplot as plt

train_losses = [4.5435, 3.6395, 3.3982]

plt.figure()
plt.plot(train_losses, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("MLM Training Loss")
plt.grid()

plt.savefig("results/train_loss.png")
plt.close()

In [27]:
with open("results/metrics.txt", "w") as f:
    f.write(f"Test Loss: {test_loss:.4f}\n")
    f.write(f"Test Accuracy: {test_acc:.4f}\n")
    f.write(f"Test F1-score: {test_f1:.4f}\n")  

In [28]:
import random
def predict_mask(text, tokenizer, model, device, mask_pos=None):
    model.eval()

    # 1️⃣ Encode text
    ids = tokenizer.encode(text)
    input_ids = torch.tensor([ids]).to(device)

    # 2️⃣ Attention mask
    attention_mask = (input_ids != tokenizer.word2id["[PAD]"]).long()

    # 3️⃣ انتخاب موقعیت mask (اگر داده نشده)
    if mask_pos is None:
        # فقط روی توکن‌های واقعی (نه CLS/SEP/PAD)
        valid_positions = [
            i for i, t in enumerate(ids)
            if t not in (
                tokenizer.word2id["[CLS]"],
                tokenizer.word2id["[SEP]"],
                tokenizer.word2id["[PAD]"]
            )
        ]
        mask_pos = random.choice(valid_positions)

    # 4️⃣ Mask کردن
    original_id = input_ids[0, mask_pos].item()
    input_ids[0, mask_pos] = tokenizer.word2id["[MASK]"]

    # 5️⃣ Forward
    with torch.no_grad():
        logits = model(input_ids, attention_mask)

    # 6️⃣ Prediction
    pred_id = torch.argmax(logits[0, mask_pos]).item()

    return {
        "masked_position": mask_pos,
        "original_token": tokenizer.id2word.get(original_id, "[UNK]"),
        "predicted_token": tokenizer.id2word.get(pred_id, "[UNK]")
    }

In [31]:
text = "یادگیری ماشین در پردازش زبان طبیعی بسیار مهم است"

result = predict_mask(text, tokenizer, model, device)

print("Original token:", result["original_token"])
print("Predicted token:", result["predicted_token"])

Original token: است
Predicted token: است


In [32]:
import json
import os

os.makedirs("artifacts", exist_ok=True)

with open("artifacts/vocab.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer.word2id, f, ensure_ascii=False, indent=2)